In [ ]:
!python -m spacy download en_core_web_sm

2023-09-13 01:47:16.840068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 01:47:19.235612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.6.0
    Uninstalling en-core-web-sm-3.6.0:
      Successfully uninstalled en-core-web-sm-3.6.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
pip install allennlp allennlp-models

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/GitHub/Male_Gaze_NLP/notebooks/

/content/gdrive/MyDrive/GitHub/Male_Gaze_NLP/notebooks


In [ ]:
!git add .
!git commit -m "Excluded long novels from list"

[main 3361c0a] Excluded long novels from list
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/SRL_Pip_modified.ipynb (64%)


In [ ]:
!git push origin main

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.48 KiB | 252.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/CoraMao1219/Male_Gaze_NLP
   31b50a8..3361c0a  main -> main


In [ ]:
import torch
device = torch.device('cuda:0')
import pickle
import re
import random
import nltk
import scipy.stats as stats

from transformers import BertTokenizer
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from srl_functions import removeProjectGutenberg_LowerCase, hasComponent, getArg0Arg1perVerb, getArg0Arg1perText

In [ ]:
date = '9_12'

## Gender List

In [17]:
masculine_terms = ["boy", "man", "gentleman", "male", "father", "son", "brother", "uncle", "nephew",
                   "grandfather", "husband", "boyfriend", "king", "prince", "emperor", "baron", "duke",
                   "earl", "sir", "mister", "godfather", "heir", "widower", "bachelor", "groom", "monsieur",
                   "friar", "wizard", "patriarch", "stepfather", "godson", "salesman", "policeman", "fireman",
                   "mailman", "stag", "sorcerer", "druid", "abbot", "knight"]

feminine_terms = ["girl", "woman", "lady", "female", "mother", "daughter", "sister", "aunt", "niece", "grandmother",
                  "wife", "girlfriend", "queen", "princess", "empress", "baroness", "duchess", "countess", "madam",
                  "miss", "godmother", "heiress", "widow", "bachelorette", "bride", "mademoiselle", "nun", "witch",
                  "matriarch", "stepmother", "goddaughter", "saleswoman", "policewoman", "firewoman", "mailwoman",
                  "doe", "heroine", "sorceress", "druidess", "abbess", "dame"]

f_typical = ["woman","women","female","females","girl","girls","she","her","herself","wife","wives","mother","mothers","miss","miss.","mrs","mrs.","lady","ladies"]
m_typical = ["man","men","male","males","boy","boys","he","him","himself","husband","husbands","father","fathers","mr","mr.","sir"]

femaleList_typical = list(set(f_typical + feminine_terms))
maleList_typical = list(set(m_typical + masculine_terms))

In [22]:
# list_Female = []
# for char in f_typical + feminine_terms:
#     if char not in list_Female:
#         list_Female.append(char)
# print(len(femaleList_typical) == len(list_Female))
# print(len(femaleList_typical))
# print(list_Female)

True
53
['woman', 'women', 'female', 'females', 'girl', 'girls', 'she', 'her', 'herself', 'wife', 'wives', 'mother', 'mothers', 'miss', 'miss.', 'mrs', 'mrs.', 'lady', 'ladies', 'daughter', 'sister', 'aunt', 'niece', 'grandmother', 'girlfriend', 'queen', 'princess', 'empress', 'baroness', 'duchess', 'countess', 'madam', 'godmother', 'heiress', 'widow', 'bachelorette', 'bride', 'mademoiselle', 'nun', 'witch', 'matriarch', 'stepmother', 'goddaughter', 'saleswoman', 'policewoman', 'firewoman', 'mailwoman', 'doe', 'heroine', 'sorceress', 'druidess', 'abbess', 'dame']


In [23]:
# list_Male = []
# for char in m_typical + masculine_terms:
#     if char not in list_Male:
#         list_Male.append(char)
# print(len(maleList_typical) == len(list_Male))
# print(len(list_Male))
# print(list_Male)

True
50
['man', 'men', 'male', 'males', 'boy', 'boys', 'he', 'him', 'himself', 'husband', 'husbands', 'father', 'fathers', 'mr', 'mr.', 'sir', 'gentleman', 'son', 'brother', 'uncle', 'nephew', 'grandfather', 'boyfriend', 'king', 'prince', 'emperor', 'baron', 'duke', 'earl', 'mister', 'godfather', 'heir', 'widower', 'bachelor', 'groom', 'monsieur', 'friar', 'wizard', 'patriarch', 'stepfather', 'godson', 'salesman', 'policeman', 'fireman', 'mailman', 'stag', 'sorcerer', 'druid', 'abbot', 'knight']


In [ ]:
nameCharList = pickle.load(open("/content/gdrive/MyDrive/NLU1012_Project/Name_results.p", 'rb'))

In [ ]:
nameCharList.keys()

dict_keys(['Emma', 'Great_Expectations', 'Jane_Eyre', 'Little_Women', 'Lolita', 'Mrs_Dalloway_in_Bond_Street', 'Pride_and_Prejudice', 'The_Great_Gatsby', 'The_Three_Musketeers', 'Villette', 'Moby-Dick', 'A_Room_with_a_View', 'The_Blue_Castle', 'The_Enchanted_April', 'Cranford', 'Twenty_Years_After', 'Down_and_Out_in_the_Magic_Kingdom', "Alice's_Adventures_in_Wonderland", 'Frankenstein', 'Middlemarch', 'A_Tale_of_Two_Cities', 'Dracula', 'The_Picture_of_Dorian_Gray', 'The_Adventures_of_Sherlock_Holmes', 'Anna_Karenina', 'The_Count_of_Monte_Cristo', 'War_and_Peace', 'The_Brothers_Karamazov', 'Metamorphosis', 'Ulysses', 'Crime_and_Punishmen', 'The_Yellow_Wallpaper', 'The_Romance_of_Lust_A_Classic_Victorian_Erotic_Novel', 'Grimms_Fairy_Tales', 'The_Adventures_of_Tom_Sawyer_Complete', 'The_Wonderful_Wizard_of_Oz', 'Adventures_of_Huckleberry_Finn', 'Anne_of_Green_Gables', 'The_Strange_Case_of_Dr_Jekyll_and_Mr_Hyde', 'Treasure_Island', 'A_Study_in_Scarlet', 'Wuthering_Heights', 'The_Scarlet_Le

In [ ]:
Cindy = ['Crime_and_Punishmen', 'The_Yellow_Wallpaper', 'The_Romance_of_Lust_A_Classic_Victorian_Erotic_Novel', 'Grimms_Fairy_Tales', 'The_Adventures_of_Tom_Sawyer_Complete', 'The_Wonderful_Wizard_of_Oz', 'Adventures_of_Huckleberry_Finn', 'Anne_of_Green_Gables', 'The_Strange_Case_of_Dr_Jekyll_and_Mr_Hyde', 'Treasure_Island', 'A_Study_in_Scarlet','Wuthering_Heights', 'The_Scarlet_Letter', 'The_War_of_the_Worlds','Uncle_Toms_Cabin', 'The_Idiot', 'Dombey_and_Son']
Bella = ['The_Time_Machine', 'Carmilla', 'Through_the_LookingGlass', 'Heart_of_Darkness', 'David_Copperfield', 'Sense_and_Sensibility', 'Oliver_Twist', 'The_Souls_of_Black_Fork', 'The_Murder_of_Roger_Ackroyd', 'Three_Men_in_a_Boat', 'Notes_from_the_Underground', 'Les_Miserables', 'The_Hound_of_the_Baskervilles', 'The_Mysterious_Affair_at_Styles', 'The_Secret_Garden', 'Lady_Audleys_Secret', 'The_Call_of_the_Wild']
Cora = ['A_Christmas_Carol_in_Prose', 'The_Prince_and_the_Pauper', 'Anthem', 'Around_the_World_in_Eighty_Days', 'The_Sun_Also_Rises', 'The_Trial', 'A_Portrait_of_the_Artist_as_a_Young_Man', 'Northanger_Abbey', 'The_Sign_of_the_Four', 'The_Jungle', 'The_Reign_of_Greed', 'The_Possessed', 'The_Wind_in_the_Willows', 'Bleak_House', 'The_Turn_of_the_Screw', 'The_Tale_of_Peter_Rabbit', 'Madame_Bovary', 'Twenty_Thousand_Leagues_under_the_Sea']
print(len(Cindy), len(Bella), len(Cora))

17 17 18


In [ ]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
novelPath_0 = '/content/gdrive/MyDrive/NLU1012_Project/novels/'
running_Name = Cindy ## change this to Bella or Cora
allResults = {}
allCheckResults = {}
all_0_1_ratio = {}
chi_square_results = {}
longNovels = ['Ulysses','Les_Miserables', 'War_and_Peace']
completedNovels = ['Emma', 'Great_Expectations', 'Jane_Eyre', 'Little_Women', 'Lolita',
                   'Mrs_Dalloway_in_Bond_Street', 'Pride_and_Prejudice', 'The_Great_Gatsby',
                   'The_Three_Musketeers', 'Villette', 'Moby-Dick', 'A_Room_with_a_View',
                   'The_Blue_Castle', 'The_Enchanted_April', 'Cranford', 'Twenty_Years_After',
                   'Down_and_Out_in_the_Magic_Kingdom', "Alice's_Adventures_in_Wonderland",
                   'Frankenstein', 'Middlemarch', 'A_Tale_of_Two_Cities', 'Dracula',
                   'The_Picture_of_Dorian_Gray', 'The_Adventures_of_Sherlock_Holmes', 'Anna_Karenina',
                   'The_Count_of_Monte_Cristo', 'The_Brothers_Karamazov']

for novel in running_Name:
    # if novel == 'Emma':
    if (novel not in completedNovels) and (novel not in longNovels):
        print(f'-----------------------{novel.upper()}-------------------------')
        path = novelPath_0 + novel + '.txt'
        result = {'female':{}, 'male':{}}
        checkResult = {'description':[], 'result':[]}
        novel_fChar = nameCharList[novel]['Female']
        novel_mChar = nameCharList[novel]['Male']
        novel_surnames = nameCharList[novel]['Surname']
        femaleList = list(set(removeProjectGutenberg_LowerCase(novel_fChar) + femaleList_typical))
        maleList = list(set(removeProjectGutenberg_LowerCase(novel_mChar) + maleList_typical))
        surnameList = removeProjectGutenberg_LowerCase(novel_surnames)
        with open(path) as f:
            text = f.read().replace('\n', ' ')
        sentences = nltk.sent_tokenize(text)
        chunk_size = 1000
        sentences_length = len(sentences)
        for i in range(0, sentences_length, chunk_size):
            chunk = sentences[i:i + chunk_size]
            result = getArg0Arg1perText(chunk, femaleList, maleList, surnameList, result, checkResult)
            print(f'Completed text from {i} to {i + chunk_size}.')
            print(f'{novel}: {result}')
        allResults[novel] = result
        allCheckResults[novel] = checkResult

        female_0_1_ratio = result['female']['ARG0']/(result['female']['ARG0']+result['female']['ARG1'])
        print(f'female_0_1_ratio: {female_0_1_ratio}')
        male_0_1_ratio = result['male']['ARG0']/(result['male']['ARG0']+result['male']['ARG1'])
        print(f'male_0_1_ratio: {male_0_1_ratio}')

        all_0_1_ratio[novel] = {'female':female_0_1_ratio, 'male':male_0_1_ratio}

        # Chi-square
        data = [[result['female']['ARG0'], result['male']['ARG0']],  # Counts for subject
            [result['female']['ARG1'], result['male']['ARG1']]]  # Counts for object

        # Perform Chi-square test
        chi2, p, dof, expected = stats.chi2_contingency(data)

        # Output the results
        print(f"Chi-Square value = {chi2}")
        print(f"P-value = {p}")
        print(f"Degrees of Freedom = {dof}")
        print(f"Expected frequencies: \n{expected}")
        chi_square_results[novel] = {'Chi-Square value':chi2,'P-value':p,'Degrees of Freedom':dof,'Expected frequencies':expected}
        print(f'DONE with {novel}')
        completedNovels.append(novel)


-----------------------CRIME_AND_PUNISHMEN-------------------------
Completed text from 0 to 1000.
Crime_and_Punishmen: {'female': {'ARG0': 393, 'ARG1': 268}, 'male': {'ARG1': 352, 'ARG0': 621}}
Completed text from 1000 to 2000.
Crime_and_Punishmen: {'female': {'ARG0': 573, 'ARG1': 441}, 'male': {'ARG1': 739, 'ARG0': 1601}}
Completed text from 2000 to 3000.
Crime_and_Punishmen: {'female': {'ARG0': 683, 'ARG1': 519}, 'male': {'ARG1': 1175, 'ARG0': 2352}}
Completed text from 3000 to 4000.
Crime_and_Punishmen: {'female': {'ARG0': 871, 'ARG1': 656}, 'male': {'ARG1': 1556, 'ARG0': 3103}}
Completed text from 4000 to 5000.
Crime_and_Punishmen: {'female': {'ARG0': 1193, 'ARG1': 894}, 'male': {'ARG1': 1930, 'ARG0': 3684}}
Completed text from 5000 to 6000.
Crime_and_Punishmen: {'female': {'ARG0': 1260, 'ARG1': 982}, 'male': {'ARG1': 2227, 'ARG0': 4323}}
Completed text from 6000 to 7000.
Crime_and_Punishmen: {'female': {'ARG0': 1614, 'ARG1': 1257}, 'male': {'ARG1': 2530, 'ARG0': 4847}}
Completed 

In [ ]:
allResults

{'Crime_and_Punishmen': {'female': {'ARG0': 2878, 'ARG1': 2253},
  'male': {'ARG1': 4015, 'ARG0': 7375}},
 'The_Yellow_Wallpaper': {'female': {'ARG1': 37, 'ARG0': 53},
  'male': {'ARG0': 74, 'ARG1': 22}},
 'The_Romance_of_Lust_A_Classic_Victorian_Erotic_Novel': {'female': {'ARG1': 2969,
   'ARG0': 4813},
  'male': {'ARG0': 1992, 'ARG1': 1041}},
 'Grimms_Fairy_Tales': {'female': {'ARG0': 2042, 'ARG1': 1196},
  'male': {'ARG0': 3257, 'ARG1': 1623}},
 'The_Adventures_of_Tom_Sawyer_Complete': {'female': {'ARG0': 626,
   'ARG1': 281},
  'male': {'ARG1': 1270, 'ARG0': 2525}},
 'The_Wonderful_Wizard_of_Oz': {'female': {'ARG0': 781, 'ARG1': 366},
  'male': {'ARG1': 448, 'ARG0': 727}},
 'Adventures_of_Huckleberry_Finn': {'female': {'ARG0': 748, 'ARG1': 392},
  'male': {'ARG1': 1504, 'ARG0': 2850}},
 'Anne_of_Green_Gables': {'female': {'ARG0': 3541, 'ARG1': 1931},
  'male': {'ARG1': 542, 'ARG0': 876}},
 'The_Strange_Case_of_Dr_Jekyll_and_Mr_Hyde': {'female': {'ARG1': 15,
   'ARG0': 22},
  'male'

In [ ]:
name = 'Cindy' ### change to your own name
path = "/content/gdrive/MyDrive/NLU1012_Project/SRL_Results/"

with open(path + f'allSRLResults_{date}_{name}.pkl', 'wb') as f:
    pickle.dump(allResults, f)

with open(path + f'allSRLCheckResults_{date}_{name}.pkl', 'wb') as f:
    pickle.dump(allCheckResults, f)

with open(path + f'allSRL0-1Ratios_{date}_{name}.pkl', 'wb') as f:
    pickle.dump(all_0_1_ratio, f)

with open(path + f'allSRLChiSquare_{date}_{name}.pkl', 'wb') as f:
    pickle.dump(chi_square_results, f)